In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from highcharts import Highchart
import wordcloud as wc
#import pygal as pg
#import altair as aa
import plotly as pl
#import bokeh as bk
#import gleam as gm
#import ggplot as gp
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
########## This code ensures that plotly plots are renderred in jupyter notebook ####
# This is also called as offline mode or something  #

# Import the necessaries libraries
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# Set notebook mode to work in offline
pyo.init_notebook_mode()

In [ ]:
# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 500
%config InlineBackend.figure_format = 'svg'

In [ ]:
df = pd.read_csv('./winemag-data-130k-v2.csv', index_col=0)
df.drop_duplicates(('description', 'title'), inplace=True)
df[pd.notnull(df.price)]
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

# Imputing missing values
for col in ('region_2', 'designation', 'taster_twitter_handle', 'taster_name', 'region_1'):
    df[col]=df[col].fillna('Unknown')
df['province'] = df['province'].fillna(df['province'].mode())
df['price'] = df['price'].fillna(df['price'].mean())

In [ ]:
data=df.copy()

In [ ]:
data.info()

In [ ]:
data.describe()

# No. of Wine Tasted According to Country

In [ ]:
df=data.copy()
plt.figure(figsize=(10,5))
sns.set(style="darkgrid")
s=sns.barplot(x=df.country.value_counts()[:15].index, y=df.country.value_counts()[:15].values)
plt.xlabel("Countries")
plt.ylabel("Number of Wine Samples")
plt.title('No. of Samples per Country - Top 15')
s.set_xticklabels(s.get_xticklabels(),rotation=45)
plt.show()

In [ ]:
##### Histogram and boxplot for points column ########
import plotly.express as px
df = data['points']
fig = px.histogram(df, x="points", y="points", nbins=50,
                   marginal="box", # box or violin, rug
                   hover_data=['points'],histnorm='probability density')
fig.show()

In [ ]:
####### Histogram and boxplot for prices column ########
import plotly.express as px
df = data['price']
fig = px.histogram(df, x="price", y="price", nbins=100,
                   marginal="box", # box or violin, rug
                   hover_data=['price'],histnorm='probability density')
fig.show()

In [ ]:
###### mean prices across countries ########
import plotly.express as px

df = data.groupby(['country']).mean().reset_index()
fig = px.choropleth(df, locations="country", locationmode='country names', 
                    color="price", # lifeExp is a column of gapminder
                    hover_name="price", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
############ mean points across countries ############
import plotly.express as px

df = data.groupby(['country']).mean().reset_index()
fig = px.choropleth(df, locations="country", locationmode='country names', 
                    color="points", # lifeExp is a column of gapminder
                    hover_name="points", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
df.isnull().sum()

In [ ]:
##### fill country column with the value 'NaN' #####
df = data.copy()
df['country']=df['country'].fillna(value=0)

In [ ]:
df = data
fig = px.treemap(df, path=['country', 'variety'], values='price')
fig.show()

In [ ]:
# idx1=df[['variety','country','price']].copy()
# idx1

In [ ]:
#### tree map for different varieties across different countries #####
# import plotly.express as px

# idx1=idx1.groupby(["country","variety"])
# idx1.head()



In [ ]:
# maximums=idx1.max()
# maximums=maximums.reset_index()
# # ig = px.treemap(maximums, path=['country','variety'], values='price',title='Most Expensive Wines per Country')
# # fig.show()
# maximums


In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS)
# iterate through the csv file 
for val in data.description: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

# Points vs Price Correlations

In [ ]:
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency

In [ ]:
df=data.copy()
df=df[['points','price']]
df

In [ ]:
#spearman's rank correlation
correlation,pval=spearmanr(df)

In [ ]:
print(f'correlation={correlation:.6f}, p-value={pval:.6f}')

NULL hypothesis, no correlation between price and value
A p-value less than 0.05 (typically ≤ 0.05) is statistically significant. It indicates strong evidence against the null hypothesis, as there is less than a 5% probability the null is correct (and the results are random). Therefore, we reject the null hypothesis, and accept the alternative hypothesis.
However, this does not mean that there is a 95% probability that the research hypothesis is true. The p-value is conditional upon the null hypothesis being true is unrelated to the truth or falsity of the research hypothesis.



In [ ]:
df.corr(method='spearman')

In [ ]:
sns.pairplot(df)

In [ ]:
fig = px.scatter_matrix(df)
fig.show()

In [ ]:
df=data.copy()
df['price_log'] = np.log(df['price'])

In [ ]:
fig=px.scatter(x=df['price'],y=df['points'],labels={'x':'Price','y':'Points'})
fig.show()

# Country Graphs

95-100 Classic: a great wine 

90-94 Outstanding: a wine of superior character and style

85-89 Very good: a wine with special qualities

80-84 Good: a solid, well-made wine

75-79 Mediocre: a drinkable wine that may have minor flaws

50-74 Not recommended

In [ ]:
# ###Data Processing for the bar graphh###
# idx=data.loc[data['points']>=95]
# idx1=idx.groupby('country').count().reset_index()
# idx1=idx1.sort_values(by=['points'],ascending=True)
# mapping_colors=name_to_color(idx1.country,0,0,0,0,0,230)
# idx1['Color']=idx1['country'].map(mapping_colors)
# idx1['norm_points']=(idx1['points']/idx1['points'].sum()*100).map('{:,.2f}'.format).astype(str)

# layout = dict(yaxis={'title':'Country'},
#              xaxis={'title':'Count of Points'},
#              title='Countries with Highest Rated Wines (95-100)')

# #####Plotting#####
# fig=go.Figure(go.Bar(x=idx1['points'],y=idx1['country'],marker_color=idx1['Color'], text=idx1['norm_points']+'%',textposition='auto',orientation='h'),layout=layout)
# fig.show()


In [ ]:
# idx=data.loc[data['price']<50]
# idx1=idx.groupby('country').count().reset_index()
# idx1

In [ ]:
top_5_countries_prw = data.groupby('country').mean().sort_values(by = ['price'],ascending = False)
top_5_countries_prw = pd.DataFrame(top_5_countries_prw.iloc[0:5,1])
top_5_countries_prw['Index'] = list(range(5))
top_5_countries_prw['Country'] = top_5_countries_prw.index.values
top_5_countries_prw.set_index(['Index'],inplace = True)
top_5_countries_prw

### Highest Rated Wines Country-Wise

In [ ]:
layout = dict(yaxis={'title':'Country'},
             xaxis={'title':'Count of Wine'},
             title='Countries with Highest Rated Wines (95-100)')
df=data.copy()
idx=df.loc[df['points']>=95]
idx=idx.sort_values(by=['points'],ascending=False)
fig=go.Figure(go.Bar(x=idx.country.value_counts().values[:15],y=idx.country.value_counts().index[:15],orientation='h')
              ,layout=layout)
fig.show()



### Highest Priced Wines Country-wise

In [ ]:
layout = dict(yaxis={'title':'Country'},
             xaxis={'title':'Count of Wine'},
             title='Countries with Highest Priced Wines >500 USD)')
df=data.copy()
idx1=df.loc[df['price']>500]
idx1=idx1.sort_values(by=['price'],ascending=False)
fig=go.Figure(go.Bar(x=idx1.country.value_counts().values[:15],y=idx1.country.value_counts().index[:15],orientation='h')
              ,layout=layout)
fig.show()



### Countries that make affordable wines and highly rated

In [ ]:
df=data.copy()
idx5=df.loc[(df['points']>=95) & (df['price']<50)]


layout = dict(yaxis={'title':'Country'},
             xaxis={'title':'Count of Wine'},
             title='Countries with Affordable and Highly Rated Wines')



fig=go.Figure(go.Bar(x=idx5.country.value_counts().values[:15],y=idx5.country.value_counts().index[:15],orientation='h')
              ,layout=layout)
fig.show()



# Countries that make costly wines and highly rate

In [ ]:
df=data.copy()
idx6=df.loc[(df['price']>500)& (df['points']>=95)]

layout = dict(yaxis={'title':'Country'},
             xaxis={'title':'Count of Wine'},
             title='Countries with Expensive and Highly Rated Wines')



fig=go.Figure(go.Bar(x=idx6.country.value_counts().values[:15],y=idx6.country.value_counts().index[:15],orientation='h')
              ,layout=layout)
fig.show()



# Wine Variety Analysis

### Grapes that produced Highest Rated Wines on Average


In [ ]:
df=data.copy()

df=df[(df['points']>=95)]
df=df.sort_values(by='points',ascending=False)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,7)})
ax=sns.barplot(x=df.variety.value_counts().index[:15],y=df.variety.value_counts().values[:15],data=df,palette="rocket")
plt.title('Grapes Producing Highest Rated Wines')
plt.xlabel('Grapes')
plt.ylabel('Count')
for item in ax.get_xticklabels():
    item.set_rotation(90)

### Grapes that produced the Most Expensive Wines (>500 USD)


In [ ]:
df2=data.copy()

df=df2[(df2['price']>500)]

In [ ]:
from numpy import median
sns.set(rc={'figure.figsize':(11.7,7)})
ax=sns.barplot(x=df.variety.value_counts().index[:15],y=df.variety.value_counts().values[:15],data=df,palette="turbo")
plt.title('Grapes Producing the Most Expensive Wines')
plt.xlabel("Grape")
plt.ylabel("Count")
for item in ax.get_xticklabels():
    item.set_rotation(90)

# Grapes that produce the Highest Rated and Expensive Wines


In [ ]:
df3=data.copy()
df3=df3.loc[(df3['price']>500) & (df3['points']>=95)]

In [ ]:
sns.set(rc={'figure.figsize':(10,7)})
ax=sns.barplot(x=df3.variety.value_counts().index[:15],y=df3.variety.value_counts().values[:15],data=df3,palette="turbo")
plt.xlabel('Grape Variety')
plt.ylabel('Count of Wine')
plt.title('Grapes Producing High Rated and Expensive Wines')
for item in ax.get_xticklabels():
    item.set_rotation(90)
    


# Grapes that produce the Highest Rated and Affordable Wines


In [ ]:
df3=data.copy()
df3=df3.loc[(df3['price']<50) & (df3['points']>=95)]

In [ ]:
sns.set(rc={'figure.figsize':(10,7)})
ax=sns.barplot(x=df3.variety.value_counts().index[:15],y=df3.variety.value_counts().values[:15],data=df3,palette="Blues_r")
plt.xlabel('Grape Variety')
plt.ylabel('Count of Wine')
plt.title('Grapes Producing High Rated and Affordable Wines')
for item in ax.get_xticklabels():
    item.set_rotation(90)
    


# Variety of Grapes


In [ ]:
####Which grape variety had the most number of samples in the dataset####
df=data.copy()
fig = go.Figure(data=[go.Pie(labels=df['variety'].value_counts().index[:15], 
                             values=df['variety'].value_counts().values[:15],hole=.3,name="Grapes")])

fig.update_layout(
    title_text="Variety of Grapes in Dataset - Top 15",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Grapes' ,font_size=20, showarrow=False)])
fig.show()

# Vineyards that  Produce Highest Rated and Expensive Wines

In [ ]:
df=data.copy()
df=df.loc[(df['price']>500) & (df['points']>=95)]

In [ ]:
fig = go.Figure(data=[go.Pie(labels=df['winery'].value_counts().index[:15], 
                             values=df['winery'].value_counts().values[:15],hole=.3,name="Grapes")])

fig.update_layout(
    title_text="Vinyards that Produce the Highest Rated and Most Expensive Wines - Top 15",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Vinyards' ,font_size=20, showarrow=False)])
fig.show()

# Price Distribution of Highly Rated Wines

In [ ]:
df=data.copy()
df['price_log'] = np.log(df['price'])

In [ ]:
df1=df.loc[(df['points']>=95)]
df1=df1.sort_values(by='price',ascending=False)
var=df1['variety'].value_counts()[:15]
# var
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# df1


In [ ]:
g = sns.boxplot(x='variety', y='price',
                  data=df1[df1.variety.isin(var.index.values)],
                 color='darkgreen')
g.set_title("Price Distribution of Highly Rated Wines by Grape - Top 10", fontsize=20)
g.set_xlabel("Grapes ", fontsize=15)
g.set_ylabel("Price Dist", fontsize=15)
g.set_xticklabels(g.get_xticklabels(),rotation=90)

interesting to see that the 3300 red blend did not make it to the high rated wines.
- Pinot noir in the intermediate category of the price distribution.

# Points Distribution of Highly Rated Wines


In [ ]:
df=data.copy()

df=df.loc[(df['variety']=='Bordeaux-style Red Blend') | (df['variety']=='Riesling') | 
          (df['variety']=='Champagne Blend') | (df['variety']=='Chardonnay') |
          (df['variety']=='Cabernet Sauvignon') | (df['variety']=='Nebbiolo') |
          (df['variety']=='Sangiovese') | (df['variety']=='Pinot Noir') |
          (df['variety']=='Red Blend') | (df['variety']=='Syrah') 
         ]


df2=df.sort_values(by='points',ascending=False)
var=df2['variety'].value_counts()


In [ ]:
g = sns.boxplot(x='variety', y='points',
                  data=df2[df2.variety.isin(var.index.values)],
                 color='darkgreen')
g.set_title("Points Distribution of Highly Rated Wines by Grape", fontsize=20)
g.set_xlabel("Grapes", fontsize=15)
g.set_ylabel("Points Dist", fontsize=15)
g.set_xticklabels(g.get_xticklabels(),rotation=90)

Clearly shows that grapes aren't the only reason why a wine is performing better. It does to an extent, but we need more information to fill in the gap

# No. of reviewers and their review count

In [ ]:
df=data.copy()
k=df.taster_name.unique()
#df=df.loc[df['taster_name']!='Unknown']
print('We have {} reviewers.'.format(k.shape[0]))
k

In [ ]:
#plotting their count 
ax=sns.barplot(x=df.taster_name.value_counts().index,y=df.taster_name.value_counts().values,data=df,palette="Blues_r")
plt.xlabel('Wines Taster Names')
plt.ylabel('Review Count ')
plt.title('Wine Tasters vs the review count')
for item in ax.get_xticklabels():
    item.set_rotation(90)

### Score Distribution Among the reviewers

In [ ]:
df=data.copy()
#df1=df.loc[(df['points']>97)]
df=df.sort_values(by='points',ascending=False)
var=df['taster_name'].value_counts()


In [ ]:
g = sns.boxplot(x='taster_name', y='points',
                  data=df[df.taster_name.isin(var.index.values)],
                 color='darkgreen')
#g = sns.swarmplot(x="taster_name", y="points", data=df[df.taster_name.isin(var.index.values)], color=".25")
g.set_title("Score Distribution of Wine Tasters", fontsize=20)
g.set_xlabel("Grapes ", fontsize=15)
g.set_ylabel("Score Distribution", fontsize=15)
g.set_xticklabels(g.get_xticklabels(),rotation=90)

### 


### Reviewers and their scores to our high rated wines
We further wanted to explore what the top 3 and bottom 3 scored different wines

In [ ]:
df1=df.loc[(df['points']>97)]
df1=df1.sort_values(by='price',ascending=False)
var=df1['variety'].value_counts()[:10]